# Baseline Models

1. Recommending songs by the same artist
2. Recommending songs from the same album
3. Recommending the most "popular" songs
___

### Imports

In [1]:
import random
import os
import csv

# from tqdm import tqdm # for scripts
from tqdm.auto import tqdm # for notebooks
tqdm.pandas()

import numpy as np
import pandas as pd

import tensorflow as tf


# Importing 2017 data
Copying cells from `load_2017_data.ipynb` notebook, which produces the following:

Loads the following csv's into **DataFrames**:
- `playlist_2017.csv` $\rightarrow$ `playlist_df`
- `tracks_2017.csv` $\rightarrow$ `track_df`
- `albums_2017.csv` $\rightarrow$ `album_df`
- `artist_2017.csv` $\rightarrow$ `artist_df`

Then, the following **dictionaries** are created:
- `playlist_dict`
- `track_dict`
- `album_dict`
- `artist_dict`

As well as the following **functions**:
- `get_playlist_feature(PID, feature)`
- `get_track_feature(URI, feature)`
- `get_album_feature(URI, feature)`
- `get_artist_feature(URI, feature)`

### Update data path as needed

In [2]:
# path to the DIRECTORY where CSV files are saved relative to this notebook
path = '../../data'
output_filepath_root = os.path.relpath(path)
print(output_filepath_root)

../../data


## Load 2017 CSVs to DFs

### Playlists

In [3]:
playlist_df = pd.read_csv(os.path.join(output_filepath_root, 'playlists_2017.csv'), index_col='pid')
playlist_df.head(5)

,name,description,modified_at,num_artists,num_albums,num_tracks,num_followers,num_edits,duration_ms,collaborative,...,track_14_album_uri,track_14_artist_uri,track_15_uri,track_15_album_uri,track_15_artist_uri,modified_at_date,modified_at_year,modified_at_month,modified_at_day,modified_at_dow
pid,,,,,,,,,,,,,,,,,,,,,
434000,Sad,NaN,1488240000,24,26,27,1,6,6081757,False,...,spotify:album:4M9Ti6t5h54aDMX4SizDfT,spotify:artist:4vVfuZfXWu18vk5Z4C7wbm,spotify:track:3yrVRdwCbEeKODZgG2mVZX,spotify:album:3SCJmoy3Z45p84IfuaM9YQ,spotify:artist:2EO56JK4txid1Pss9GVbOL,2017-02-28,2017,2,28,1
434001,pb&j,faves tbh,1487808000,35,38,39,1,9,8959761,False,...,spotify:album:6deiaArbeoqp1xPEGdEKp1,spotify:artist:0L8ExT028jH3ddEcZwqJJ5,spotify:track:5E30LdtzQTGqRvNd7l6kG5,spotify:album:18iFxjZugvKhuNNMbLjZJF,spotify:artist:77SW9BnxLY8rJ0RciFqkHh,2017-02-23,2017,2,23,3
434004,Whatever,NaN,1506816000,36,57,79,1,46,18874072,False,...,spotify:album:2Tyx5dLhHYkx6zeAdVaTzN,spotify:artist:4LLpKhyESsyAXpc4laK94U,spotify:track:0htTZnlk6okQ1HIq4EvFQ6,spotify:album:6liIoWzpvrff945pUI7fHt,spotify:artist:02kJSzxNuaWGqwubyUba0Z,2017-10-01,2017,10,1,6
434005,roadtrip,NaN,1492905600,30,35,70,1,18,15696608,False,...,spotify:album:6DwdzG4UGYLxJ2p7bd483v,spotify:artist:2Q0MyH5YMI5HPQjFjlq5g3,spotify:track:4XvKjZWIqsHvvza89lMTAH,spotify:album:6izXZb0VGaUHqm5GaXq4YC,spotify:artist:5IXalAOiV9I8LgLMGZydmt,2017-04-23,2017,4,23,6
434007,Cumbias,NaN,1507507200,23,42,50,1,21,10255699,False,...,spotify:album:392uNMyh5D6fqkBS385XJd,spotify:artist:3zzeZVLuOeetfimOd4k8rE,spotify:track:1yOLrH7nF0R7MWDuuva6va,spotify:album:2rXbAorimO8C06RkqS2oq5,spotify:artist:0OhiQFSqbnnmB52NWEpsO5,2017-10-09,2017,10,9,0


### Tracks

In [4]:
track_df = pd.read_csv(os.path.join(output_filepath_root, 'tracks_2017.csv'), index_col='track_uri')
track_df.head(5)

,index,track_name,album_name,album_uri,artist_name,artist_uri,duration_ms,appears_in_count,popularity
track_uri,,,,,,,,,
spotify:track:6SbAbLqAWf2tnTdUy6Gmm5,0,FUCKING BEST SONG EVERRR,FUCKING BEST SONG EVERRR,spotify:album:1hmvZb81DAeTx67G1FaTjZ,Wallpaper.,spotify:artist:6NMcnx3vKGSAeqSMbySlpw,217800,53,53
spotify:track:1MvpPH6BTP3IrLnTjEA2gw,1,#STUPiDFACEDD,#STUPiDFACEDD,spotify:album:1c7wJm9mghFyIKnQJOobW8,Wallpaper.,spotify:artist:6NMcnx3vKGSAeqSMbySlpw,184026,39,39
spotify:track:5rgy6ghBq1eRApCkeUdJXf,2,We Are Young (feat. Janelle Monáe) - feat. Jan...,Some Nights,spotify:album:7m7F7SQ3BXvIpvOgjW51Gp,fun.,spotify:artist:5nCi3BB41mBaMH9gfr6Su0,250626,1276,1276
spotify:track:07dYGGSrzPeg6a3KZjWX65,3,Boyfriend,Believe,spotify:album:7BWK3eXcbAdwYeulyQj5Kw,Justin Bieber,spotify:artist:1uNFoZAHBGtllmzznpCI3s,171333,704,704
spotify:track:1NpW5kyvO4XrNJ3rnfcNy3,4,Wild Ones (feat. Sia),Wild Ones,spotify:album:7eLwoxxWs6lfkVYJGkGNbk,Flo Rida,spotify:artist:0jnsk9HBra6NMjO2oANoPY,232946,952,952


### Albums

In [5]:
album_df = pd.read_csv(os.path.join(output_filepath_root, 'albums_2017.csv'), index_col='album_uri')
album_df.head(5)

,album_name,artist_name
album_uri,,
spotify:album:1hmvZb81DAeTx67G1FaTjZ,FUCKING BEST SONG EVERRR,Wallpaper.
spotify:album:1c7wJm9mghFyIKnQJOobW8,#STUPiDFACEDD,Wallpaper.
spotify:album:7m7F7SQ3BXvIpvOgjW51Gp,Some Nights,fun.
spotify:album:7BWK3eXcbAdwYeulyQj5Kw,Believe,Justin Bieber
spotify:album:7eLwoxxWs6lfkVYJGkGNbk,Wild Ones,Flo Rida


### Artists

In [6]:
artist_df = pd.read_csv(os.path.join(output_filepath_root, 'artists_2017.csv'), index_col='artist_uri')
artist_df.head(5)

,artist_name
artist_uri,
spotify:artist:6NMcnx3vKGSAeqSMbySlpw,Wallpaper.
spotify:artist:5nCi3BB41mBaMH9gfr6Su0,fun.
spotify:artist:1uNFoZAHBGtllmzznpCI3s,Justin Bieber
spotify:artist:0jnsk9HBra6NMjO2oANoPY,Flo Rida
spotify:artist:4AK6F7OLvEQ5QYCBNiQWHq,One Direction


## Create Dictionaries

### Playlists

In [7]:
playlist_dict = playlist_df.to_dict()

In [8]:
def get_playlist_feature(pid, feature='name', dictionary=playlist_dict):
    """
    Given a playlist PID (as a integer) and the playlist dictionary, return the requested feature.
    
    Features include:
        - 'name'
        - 'description'
        - 'modified_at'
        - 'modified_at_date'
        - 'modified_at_year'
        - 'modified_at_month'
        - 'modified_at_day'
        - 'modified_at_dow'
        - 'num_artists'
        - 'num_albums'
        - 'num_tracks'
        - 'num_followers'
        - 'num_edits'
        - 'duration_ms'
        - 'collaborative'
        - 'track_X_uri'
        - 'track_X_album_uri'
        - 'track_X_artist_uri'
    """
    # Convert PID to integer
    try:
        pid = int(pid)
    except Exception as e:
        return e
    # Lookup in dictionary
    try:
        return dictionary[feature][pid]
    except Exception as e:
        return e

In [9]:
get_playlist_feature('434004', 'name')

'Whatever'

### Tracks

In [10]:
track_dict = track_df.to_dict()

In [11]:
def get_track_feature(uri, feature='track_name', dictionary=track_dict):
    """
    Given a track uri (as a string) and the track dictionary, return the requested feature.
    
    Features include:
        - 'track_name'
        - 'album_name'
        - 'album_uri'
        - 'artist_name'
        - 'artist_uri'
        - 'duration_ms'
    """
    try:
        return dictionary[feature][uri]
    except Exception as e:
        return e

In [12]:
get_track_feature('spotify:track:5rgy6ghBq1eRApCkeUdJXf', 'album_name')

'Some Nights'

### Albums

In [13]:
album_dict = album_df.to_dict()

In [14]:
def get_album_feature(uri, feature='album_name', dictionary=album_dict):
    """
    Given an album uri (as a string) and the album dictionary, return the requested feature.
    
    Features include:
        - 'album_name'
        - 'artist_name'
    """
    try:
        return dictionary[feature][uri]
    except Exception as e:
        return e

In [15]:
get_album_feature('spotify:album:7BWK3eXcbAdwYeulyQj5Kw', 'artist_name')

'Justin Bieber'

### Artists

In [16]:
artist_dict = artist_df.to_dict()

In [17]:
def get_artist_feature(uri, feature='artist_name', dictionary=artist_dict):
    """
    Given an artist uri (as a string) and the artist dictionary, return the requested feature.
    
    Features include:
        - 'artist_name'
    """
    try:
        return dictionary[feature][uri]
    except Exception as e:
        return e

In [18]:
get_artist_feature('spotify:artist:1uNFoZAHBGtllmzznpCI3s')

'Justin Bieber'

In [19]:
track_uri_cols = [col for col in playlist_df.columns
                  if 'uri' in col
                  and 'album' not in col
                  and 'artist' not in col]
track_uri_cols

['track_1_uri',
 'track_2_uri',
 'track_3_uri',
 'track_4_uri',
 'track_5_uri',
 'track_6_uri',
 'track_7_uri',
 'track_8_uri',
 'track_9_uri',
 'track_10_uri',
 'track_11_uri',
 'track_12_uri',
 'track_13_uri',
 'track_14_uri',
 'track_15_uri']

___
# #1. Artist Tracks

In [20]:
track_df.head(10)

,index,track_name,album_name,album_uri,artist_name,artist_uri,duration_ms,appears_in_count,popularity
track_uri,,,,,,,,,
spotify:track:6SbAbLqAWf2tnTdUy6Gmm5,0,FUCKING BEST SONG EVERRR,FUCKING BEST SONG EVERRR,spotify:album:1hmvZb81DAeTx67G1FaTjZ,Wallpaper.,spotify:artist:6NMcnx3vKGSAeqSMbySlpw,217800,53,53
spotify:track:1MvpPH6BTP3IrLnTjEA2gw,1,#STUPiDFACEDD,#STUPiDFACEDD,spotify:album:1c7wJm9mghFyIKnQJOobW8,Wallpaper.,spotify:artist:6NMcnx3vKGSAeqSMbySlpw,184026,39,39
spotify:track:5rgy6ghBq1eRApCkeUdJXf,2,We Are Young (feat. Janelle Monáe) - feat. Jan...,Some Nights,spotify:album:7m7F7SQ3BXvIpvOgjW51Gp,fun.,spotify:artist:5nCi3BB41mBaMH9gfr6Su0,250626,1276,1276
spotify:track:07dYGGSrzPeg6a3KZjWX65,3,Boyfriend,Believe,spotify:album:7BWK3eXcbAdwYeulyQj5Kw,Justin Bieber,spotify:artist:1uNFoZAHBGtllmzznpCI3s,171333,704,704
spotify:track:1NpW5kyvO4XrNJ3rnfcNy3,4,Wild Ones (feat. Sia),Wild Ones,spotify:album:7eLwoxxWs6lfkVYJGkGNbk,Flo Rida,spotify:artist:0jnsk9HBra6NMjO2oANoPY,232946,952,952
spotify:track:3SPDQfj2UfWq6A2NllZnzn,5,What Makes You Beautiful,Up All Night,spotify:album:53DrBEDi1AvhWOtCdljUiu,One Direction,spotify:artist:4AK6F7OLvEQ5QYCBNiQWHq,198053,677,677
spotify:track:7EOYLyD4pP4YJVgoVeKUQ1,6,Feel So Close - Radio Edit,18 Months,spotify:album:4TjPTVBmdyaowp08j2toa0,Calvin Harris,spotify:artist:7CajNmpbOovFoOoasH2HaY,206413,1109,1109
spotify:track:67WTwafOMgegV6ABnBQxcE,7,Some Nights,Some Nights,spotify:album:7m7F7SQ3BXvIpvOgjW51Gp,fun.,spotify:artist:5nCi3BB41mBaMH9gfr6Su0,277040,1100,1100
spotify:track:124NFj84ppZ5pAxTuVQYCQ,8,Take Care,Take Care,spotify:album:6X1x82kppWZmDzlXXK3y3q,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,277386,1255,1255


In [21]:
track_uri = 'spotify:track:67WTwafOMgegV6ABnBQxcE'

In [22]:
art_uri = get_track_feature(track_uri, feature='artist_uri')

In [44]:
track_df[track_df.artist_uri == art_uri].reset_index().sample()['track_uri']

28    spotify:track:4LKF7YGKEdEgZQ2IfflARi
Name: track_uri, dtype: object

# #2. Tracks from Same Album

In [24]:
alb_uri = get_track_feature(track_uri, feature='album_uri')
track_df[track_df.album_uri == alb_uri].sample()

,index,track_name,album_name,album_uri,artist_name,artist_uri,duration_ms,appears_in_count,popularity
track_uri,,,,,,,,,
spotify:track:7tgVDNjO3zaVWifN1vhhGJ,25994,All Alone,Some Nights,spotify:album:7m7F7SQ3BXvIpvOgjW51Gp,fun.,spotify:artist:5nCi3BB41mBaMH9gfr6Su0,183773,45,45


# #3. Popular Songs

In [25]:
track_counts_dict = {}
for col in track_uri_cols:
    for uri in playlist_df[col].tolist():
        if uri not in track_counts_dict.keys():
            track_counts_dict.update({uri: 1})
        else:
            track_counts_dict.update({uri: track_counts_dict.get(uri) + 1})

track_counts_dict

{'spotify:track:5JuA3wlm0kn7IHfbeHV0i6': 3000,
 'spotify:track:6y6jbcPG4Yn3Du4moXaenr': 2207,
 'spotify:track:6JG0qhINKVwiHxqN85j7RG': 1022,
 'spotify:track:2M4K2cYkZHN7OMJSZhPa5i': 7,
 'spotify:track:67RNKBsqRiyMOtor2RxfWR': 48,
 'spotify:track:5qqabIl2vWzo9ApSC317sa': 2632,
 'spotify:track:4KTtYhxFtFL7mBwnjkKfLm': 1325,
 'spotify:track:1bSpwPhAxZwlR2enJJsv7U': 212,
 'spotify:track:4eYaDRhiL5iesFp2EuoODr': 319,
 'spotify:track:1kQqiC1rS1FiuVpeBKN0QN': 188,
 'spotify:track:2gxHiPW79a1zO5sWOtza3g': 42,
 'spotify:track:5zBqPjh3vfttJScml99szD': 685,
 'spotify:track:1cK4JEQqeIc6nQONq7hyYZ': 190,
 'spotify:track:5IC7ebXWMbJDmyZBlmJ8DK': 1,
 'spotify:track:4tXzPOr4UaBoA7k7pnpJI1': 533,
 'spotify:track:5UWwZ5lm5PKu6eKsHAGxOk': 1039,
 'spotify:track:5bcfWAHLoMfGBdbeyoMaH3': 758,
 'spotify:track:6H75Ki32BRrJFQeDNR05s1': 81,
 'spotify:track:34Jeq8ktRzyOW1uw40InVy': 1034,
 'spotify:track:5SqSckut3FcoQKmGkMWgp1': 2868,
 'spotify:track:4gmmRb6bZJffOOiww1JGTO': 3720,
 'spotify:track:3rq5w4bQGigXOfdN

In [26]:
# Preview which songs were most popular to do a sense check
sor = sorted(track_counts_dict, key=lambda x: track_counts_dict.get(x), reverse=True)
for i in sor[:20]:
    print(i, get_track_feature(i), track_counts_dict[i])

0 '0' 198820
spotify:track:7KXjTSCq5nL1LoYtL7XAwS HUMBLE. 11998
spotify:track:3a1lNhkSLSkpJE4MSHpDu9 Congratulations 10898
spotify:track:152lZdxL1OR0ZMW6KquMif Location 9743
spotify:track:7yyRTcZmCiyzzJlNzGC9Ol Broccoli (feat. Lil Yachty) 9697
spotify:track:2EEeOnHehOozLq4aS0n6SL iSpy (feat. Lil Yachty) 9619
spotify:track:7GX5flRQZVHRAGd6B4TmDO XO TOUR Llif3 9506
spotify:track:4Km5HrUvYTaSUfiSGPJeQR Bad and Boujee (feat. Lil Uzi Vert) 9226
spotify:track:0SGkqnVQo9KPytSri1H6cF Bounce Back 9056
spotify:track:7BKLCZ1jbUBVqRi2FVlTVw Closer 8865
spotify:track:5hTpBe8h35rJ67eAWHQsJx Caroline 8426
spotify:track:7qiZfU4dY1lWllzX7mPBI3 Shape of You 8331
spotify:track:0VgkVdmE4gld66l8iyGjgx Mask Off 8134
spotify:track:343YBumqHu19cGoGARUTsd Fake Love 7751
spotify:track:5CtI0qwDJkDQGwXD1H1cLb Despacito - Remix 7725
spotify:track:6gBFPUFcJLzWGx4lenP6h2 goosebumps 7696
spotify:track:3DXncPQOG4VBw3QHh3S817 I'm the One 7585
spotify:track:1xznGGDReH1oQq0xzbwXa3 One Dance 7519
spotify:track:3kxfsdsCpFg

In [27]:
track_df.reset_index().head()

,track_uri,index,track_name,album_name,album_uri,artist_name,artist_uri,duration_ms,appears_in_count,popularity
0,spotify:track:6SbAbLqAWf2tnTdUy6Gmm5,0,FUCKING BEST SONG EVERRR,FUCKING BEST SONG EVERRR,spotify:album:1hmvZb81DAeTx67G1FaTjZ,Wallpaper.,spotify:artist:6NMcnx3vKGSAeqSMbySlpw,217800,53,53
1,spotify:track:1MvpPH6BTP3IrLnTjEA2gw,1,#STUPiDFACEDD,#STUPiDFACEDD,spotify:album:1c7wJm9mghFyIKnQJOobW8,Wallpaper.,spotify:artist:6NMcnx3vKGSAeqSMbySlpw,184026,39,39
2,spotify:track:5rgy6ghBq1eRApCkeUdJXf,2,We Are Young (feat. Janelle Monáe) - feat. Jan...,Some Nights,spotify:album:7m7F7SQ3BXvIpvOgjW51Gp,fun.,spotify:artist:5nCi3BB41mBaMH9gfr6Su0,250626,1276,1276
3,spotify:track:07dYGGSrzPeg6a3KZjWX65,3,Boyfriend,Believe,spotify:album:7BWK3eXcbAdwYeulyQj5Kw,Justin Bieber,spotify:artist:1uNFoZAHBGtllmzznpCI3s,171333,704,704
4,spotify:track:1NpW5kyvO4XrNJ3rnfcNy3,4,Wild Ones (feat. Sia),Wild Ones,spotify:album:7eLwoxxWs6lfkVYJGkGNbk,Flo Rida,spotify:artist:0jnsk9HBra6NMjO2oANoPY,232946,952,952


In [28]:
track_df.reset_index().apply(lambda row: track_counts_dict[row.track_uri], axis=1)

0           53
1           39
2         1276
3          704
4          952
          ... 
645797       1
645798       1
645799       1
645800       1
645801       1
Length: 645802, dtype: int64

In [29]:
track_df.shape

(645802, 9)

In [30]:
track_df.reset_index(inplace=True)
track_df['popularity'] = track_df.reset_index().progress_apply(lambda row: int(track_counts_dict[row.track_uri]), axis=1)
track_df.set_index('track_uri', inplace=True)
track_df.head()

  0%|          | 0/645802 [00:00<?, ?it/s]

,index,track_name,album_name,album_uri,artist_name,artist_uri,duration_ms,appears_in_count,popularity
track_uri,,,,,,,,,
spotify:track:6SbAbLqAWf2tnTdUy6Gmm5,0,FUCKING BEST SONG EVERRR,FUCKING BEST SONG EVERRR,spotify:album:1hmvZb81DAeTx67G1FaTjZ,Wallpaper.,spotify:artist:6NMcnx3vKGSAeqSMbySlpw,217800,53,53
spotify:track:1MvpPH6BTP3IrLnTjEA2gw,1,#STUPiDFACEDD,#STUPiDFACEDD,spotify:album:1c7wJm9mghFyIKnQJOobW8,Wallpaper.,spotify:artist:6NMcnx3vKGSAeqSMbySlpw,184026,39,39
spotify:track:5rgy6ghBq1eRApCkeUdJXf,2,We Are Young (feat. Janelle Monáe) - feat. Jan...,Some Nights,spotify:album:7m7F7SQ3BXvIpvOgjW51Gp,fun.,spotify:artist:5nCi3BB41mBaMH9gfr6Su0,250626,1276,1276
spotify:track:07dYGGSrzPeg6a3KZjWX65,3,Boyfriend,Believe,spotify:album:7BWK3eXcbAdwYeulyQj5Kw,Justin Bieber,spotify:artist:1uNFoZAHBGtllmzznpCI3s,171333,704,704
spotify:track:1NpW5kyvO4XrNJ3rnfcNy3,4,Wild Ones (feat. Sia),Wild Ones,spotify:album:7eLwoxxWs6lfkVYJGkGNbk,Flo Rida,spotify:artist:0jnsk9HBra6NMjO2oANoPY,232946,952,952


In [31]:
track_df.sort_values(by='popularity', ascending=False).head(20)

,index,track_name,album_name,album_uri,artist_name,artist_uri,duration_ms,appears_in_count,popularity
track_uri,,,,,,,,,
spotify:track:7KXjTSCq5nL1LoYtL7XAwS,1330,HUMBLE.,DAMN.,spotify:album:4eLPsYPBmXABThSJ821sqY,Kendrick Lamar,spotify:artist:2YZyLoL8N0Wb9xBt1NhZWg,177000,11998,11998
spotify:track:3a1lNhkSLSkpJE4MSHpDu9,741,Congratulations,Stoney,spotify:album:5s0rmjP8XOPhP6HhqOhuyC,Post Malone,spotify:artist:246dkjvS1zLTtiykXe5h60,220293,10898,10898
spotify:track:152lZdxL1OR0ZMW6KquMif,749,Location,American Teen,spotify:album:6kf46HbnYCZzP6rjvQHYzg,Khalid,spotify:artist:6LuN9FCkKOj5PcnpouEgny,219080,9743,9743
spotify:track:7yyRTcZmCiyzzJlNzGC9Ol,341,Broccoli (feat. Lil Yachty),Big Baby DRAM,spotify:album:0NrZHZ0y5kTO8EHliuUUca,DRAM,spotify:artist:5M0lbkGluOPXLeFjApw8r8,225205,9697,9697
spotify:track:2EEeOnHehOozLq4aS0n6SL,274,iSpy (feat. Lil Yachty),iSpy (feat. Lil Yachty),spotify:album:15qLmAABocTCRBSbWWqL7K,KYLE,spotify:artist:4qBgvVog0wzW75IQ48mU7v,253106,9619,9619
spotify:track:7GX5flRQZVHRAGd6B4TmDO,290,XO TOUR Llif3,Luv Is Rage 2,spotify:album:733e1ZfktLSwj96X5rsMeE,Lil Uzi Vert,spotify:artist:4O15NlyKLIASxsJ0PrXPfz,182706,9506,9506
spotify:track:4Km5HrUvYTaSUfiSGPJeQR,2935,Bad and Boujee (feat. Lil Uzi Vert),Culture,spotify:album:2AvupjUeMnSffKEV05x222,Migos,spotify:artist:6oMuImdp5ZcFhWP0ESe6mG,343150,9226,9226
spotify:track:0SGkqnVQo9KPytSri1H6cF,1301,Bounce Back,I Decided.,spotify:album:0XAIjjN5qxViVS0Y5fYkar,Big Sean,spotify:artist:0c173mlxpT3dSFRgMO8XPh,222360,9056,9056
spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,349,Closer,Closer,spotify:album:0rSLgV8p5FzfnqlEk4GzxE,The Chainsmokers,spotify:artist:69GGBxA162lTqCwzJG5jLp,244960,8865,8865


___
# #1a. Artist's Tracks by Popularity

In [32]:
track_df.sort_values(by='popularity', ascending=False)[track_df.artist_uri == art_uri].head()

/var/folders/31/90742r550bgg33_k6yzk4c9m0000gn/T/ipykernel_40660/1423605902.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  track_df.sort_values(by='popularity', ascending=False)[track_df.artist_uri == art_uri].head()


,index,track_name,album_name,album_uri,artist_name,artist_uri,duration_ms,appears_in_count,popularity
track_uri,,,,,,,,,
spotify:track:5rgy6ghBq1eRApCkeUdJXf,2,We Are Young (feat. Janelle Monáe) - feat. Jan...,Some Nights,spotify:album:7m7F7SQ3BXvIpvOgjW51Gp,fun.,spotify:artist:5nCi3BB41mBaMH9gfr6Su0,250626,1276,1276
spotify:track:67WTwafOMgegV6ABnBQxcE,7,Some Nights,Some Nights,spotify:album:7m7F7SQ3BXvIpvOgjW51Gp,fun.,spotify:artist:5nCi3BB41mBaMH9gfr6Su0,277040,1100,1100
spotify:track:5xZjfKSfTooDP2rcAgjxlH,2101,Carry On,Some Nights,spotify:album:7m7F7SQ3BXvIpvOgjW51Gp,fun.,spotify:artist:5nCi3BB41mBaMH9gfr6Su0,278373,526,526
spotify:track:1Y5wYXgX9dHwSKZdQNHEhV,14280,The Gambler,Aim and Ignite,spotify:album:68Q6uG6exOAUqF4Q7aNUEY,fun.,spotify:artist:5nCi3BB41mBaMH9gfr6Su0,251266,103,103
spotify:track:4orHVYvdG5v4G4bmp2Lwdg,20311,Sight Of The Sun,Sight Of The Sun,spotify:album:1yAMkQJYPLZY7kY4DXHjpl,fun.,spotify:artist:5nCi3BB41mBaMH9gfr6Su0,210373,82,82


In [40]:
track_df.sort_values(by='popularity', ascending=False)[track_df.artist_uri == art_uri].reset_index()['track_uri'].to_list()

/var/folders/31/90742r550bgg33_k6yzk4c9m0000gn/T/ipykernel_40660/3510573824.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  track_df.sort_values(by='popularity', ascending=False)[track_df.artist_uri == art_uri].reset_index()['track_uri'].to_list()


['spotify:track:5rgy6ghBq1eRApCkeUdJXf',
 'spotify:track:67WTwafOMgegV6ABnBQxcE',
 'spotify:track:5xZjfKSfTooDP2rcAgjxlH',
 'spotify:track:1Y5wYXgX9dHwSKZdQNHEhV',
 'spotify:track:4orHVYvdG5v4G4bmp2Lwdg',
 'spotify:track:5tbz0VnVwfF3jubvjDGdHo',
 'spotify:track:1TQyRzRnGJflBhUHAwtk1x',
 'spotify:track:2bEWRMWfH5n8QEuXgVlcxf',
 'spotify:track:6rcA2JxFUmlFw4GDLITcvj',
 'spotify:track:4NfK33ReDnc5qJluvErGnF',
 'spotify:track:7tgVDNjO3zaVWifN1vhhGJ',
 'spotify:track:0SHPEXmOSJL4Og3aJluoKd',
 'spotify:track:50g0it8QsUikGOHTEOaNXC',
 'spotify:track:5pxGxR8ltHAluRyn9FSKOh',
 'spotify:track:1SGLgGAv1W2CRF9sovnJXu',
 'spotify:track:6qKHDdDHy2GfAQC8ansb2y',
 'spotify:track:0SEWDKbdjKbADHZjDsiXOc',
 'spotify:track:3I7bOKGX8oxIQHFThQxtH4',
 'spotify:track:4HBeHxm71PBKrc6MGtnAaZ',
 'spotify:track:2rrtkm2zqQQ6CzP1A6dm3q',
 'spotify:track:0OdgyUig3oQTf96PcKJDZG',
 'spotify:track:45v5HffSapKHrgSDIX8x8I',
 'spotify:track:5KXQ15NLzgDKTWE8Cu18NJ',
 'spotify:track:2WmF4SAbX81fLDXhYsAMf9',
 'spotify:track:

___
# #2a. Tracks from Same Album by Popularity

In [34]:
track_uri = 'spotify:track:07dYGGSrzPeg6a3KZjWX65'
alb_uri = get_track_feature(track_uri, feature='album_uri')
track_df[track_df.album_uri == alb_uri].sample()

track_df.sort_values(by='popularity', ascending=False)[track_df.album_uri == alb_uri].head()

/var/folders/31/90742r550bgg33_k6yzk4c9m0000gn/T/ipykernel_40660/464665770.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  track_df.sort_values(by='popularity', ascending=False)[track_df.album_uri == alb_uri].head()


,index,track_name,album_name,album_uri,artist_name,artist_uri,duration_ms,appears_in_count,popularity
track_uri,,,,,,,,,
spotify:track:07dYGGSrzPeg6a3KZjWX65,3,Boyfriend,Believe,spotify:album:7BWK3eXcbAdwYeulyQj5Kw,Justin Bieber,spotify:artist:1uNFoZAHBGtllmzznpCI3s,171333,704,704
spotify:track:4rvtlpAeDuOn4baUJPCEhy,10166,As Long As You Love Me,Believe,spotify:album:7BWK3eXcbAdwYeulyQj5Kw,Justin Bieber,spotify:artist:1uNFoZAHBGtllmzznpCI3s,229466,549,549
spotify:track:0KTsmr6JOuhxZuiXUha1xC,10167,Beauty And A Beat,Believe,spotify:album:7BWK3eXcbAdwYeulyQj5Kw,Justin Bieber,spotify:artist:1uNFoZAHBGtllmzznpCI3s,227986,529,529
spotify:track:45qSLEJKJTwKTSBOa8LxPY,36687,All Around The World,Believe,spotify:album:7BWK3eXcbAdwYeulyQj5Kw,Justin Bieber,spotify:artist:1uNFoZAHBGtllmzznpCI3s,244546,173,173
spotify:track:6RgSSndg8PUH2UcaCgia5V,30700,Die In Your Arms,Believe,spotify:album:7BWK3eXcbAdwYeulyQj5Kw,Justin Bieber,spotify:artist:1uNFoZAHBGtllmzznpCI3s,237293,134,134


In [35]:
track_df.loc['spotify:track:5rgy6ghBq1eRApCkeUdJXf']

index                                                               2
track_name          We Are Young (feat. Janelle Monáe) - feat. Jan...
album_name                                                Some Nights
album_uri                        spotify:album:7m7F7SQ3BXvIpvOgjW51Gp
artist_name                                                      fun.
artist_uri                      spotify:artist:5nCi3BB41mBaMH9gfr6Su0
duration_ms                                                    250626
appears_in_count                                                 1276
popularity                                                       1276
Name: spotify:track:5rgy6ghBq1eRApCkeUdJXf, dtype: object